In [1]:
import ee
import datetime
import os
import itertools
import sys

from pprint import pprint
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import geemap

import subprocess
from subprocess import PIPE

In [2]:
ee.Initialize()

### Define basic parameters

In [3]:
# define the periods to perform temporal check
Check_len = 3
Check_wieght = [3,1,1]
Check_threshold = 4

# define the number of iteration
Check_iteration_num = 10

In [4]:
# define the names of each period
year_range = list(f'{i[0]}_{i[1]}' for i in zip(range(1990,2018,3),range(1992,2020,3)))

In [5]:
year_range

['1990_1992',
 '1993_1995',
 '1996_1998',
 '1999_2001',
 '2002_2004',
 '2005_2007',
 '2008_2010',
 '2011_2013',
 '2014_2016',
 '2017_2019']

### Import Classified_random_img

In [6]:
# get the classified_random_imgs and sum them up for each period
Classified_imgs = [ee.ImageCollection(f"users/wangjinzhulala/North_China_Plain_Python/classification_img/Random_{year}")
                     .sum()
                     .gte(9)
                     .set('name',year)
                  for year in year_range]

In [7]:
def Temporal_check(mode,in_tifs,weights,threshold):
    
    # get the name of the first img as the property for return img
    name = in_tifs[0].get('name')

    # zip classified_random_sum with weights
    img_multiply = list(zip(in_tifs,weights))

    
    
    if mode == 'forward':
        
        # multiply each classified_sum_img with check_weight 
        # and sum 3-periods together
        sum_tif = ee.ImageCollection([i[0].multiply(i[1]) for i in img_multiply]).sum()
        
        # thoes pixel that GREATER THAN are built-up pixel
        sum_constrained = sum_tif.gt(threshold).set('name',name)
        
    elif mode == 'backward':
                
        #_______________________Get theose pixel that was transformed from built to non-built______________
        
        # remap the last year, so the pixel value changed (built-->0; non-built -->weight)
        last_year_remap    = img_multiply[0][0].remap([0,1],[img_multiply[0][1],0])
        
        # multiply the reset img with weight
        sum_rest_img_multiply = ee.ImageCollection([i[0].multiply(i[1]) for i in img_multiply[1:]]).sum()
        
        # add sum_rest_img_multiply to last_year_remap, and theose pixel with value GREATER THAN threshold
        # are incorrectly classified as non-built, which should be corretd to built pixel
        Incorrect_non_built = sum_rest_img_multiply.add(last_year_remap)\
                                                   .gt(threshold)\
                                                   .remap([0,1],[0,2])
        
        # add Incorrect_non_built to the last year img, the built-pixel should have values of [1|2]
        sum_tif = Incorrect_non_built.add(img_multiply[0][0])
        
        # thoes pixel that LESS THAN are built-up pixel
        sum_constrained = sum_tif.eq(1).Or(sum_tif.eq(2))
    else:
        print("Please provide a correct mode ['forward'|'backward']")
    
    return sum_constrained

In [8]:
def Forward_backward(temporal_check_len,in_imgs):
    
    #_______________________________Forward temporal check________________________________________________
    
    forward_chunks  = [in_imgs[i:i + temporal_check_len] for i in range(len(in_imgs) - temporal_check_len + 1)]
    # perform the forward temporal check
    forward_tif     = [Temporal_check('forward',chunk,Check_wieght,Check_threshold) for chunk in forward_chunks]
    forwar_backward = forward_tif + Classified_imgs[len(forward_tif):]
    
    
#     #_______________________________Backward temporal check________________________________________________
    
#     # combie the forward_checked tifs with unchecked imgs in the tail, and reverse the list order
#     backward_tifs = forward_tif + Classified_imgs[len(forward_tif):]
#     backward_tifs_reverse = backward_tifs[::-1]
 
#     # slice the backward_tifs into chunks with the length of temporal_check_len
#     backward_chunks = [backward_tifs_reverse[i:i + temporal_check_len] for i in range(len(in_imgs) - temporal_check_len + 1)]
#     # perform the backward temporal check
#     backward_tif    = [Temporal_check('backward',chunk,Check_wieght,Check_threshold) for chunk in backward_chunks]

    
    
#     #___________________________Add Forward % Backward checked img together___________________________________
    
#     # reorder the backward_tif so tifs to ensure an increased time order
#     re_order_tif = backward_tif[::-1]
    
#     forwar_backward = forward_tif[:temporal_check_len-1] + re_order_tif
    
    return forwar_backward

In [9]:
Iter_temporal_check_instaces = {}

for it in range(Check_iteration_num):

    if it == 0:
        
        in_imgs = Classified_imgs
        
        forward_backward_checked =  Forward_backward(Check_len,in_imgs)
        forward_backward_with_iteration = [ee.Image(img).set('iteration',it) for img in forward_backward_checked]
        
        Iter_temporal_check_instaces[0]  = forward_backward_with_iteration
        
        
    else:
        
        in_imgs = Iter_temporal_check_instaces[it-1]
        
        forward_backward_checked =  Forward_backward(Check_len,in_imgs)
        forward_backward_with_iteration = [ee.Image(img).set('iteration',it) for img in forward_backward_checked]
        
        Iter_temporal_check_instaces[it]  = forward_backward_with_iteration

In [10]:
Iter_temporal_check_instaces[9][0].getInfo()

{'type': 'Image',
 'bands': [{'id': 'b1',
   'data_type': {'type': 'PixelType', 'precision': 'int', 'min': 0, 'max': 1},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]}],
 'properties': {'name': '1990_1992', 'iteration': 9}}

In [12]:
Map = geemap.Map()
Map.setCenter(115.4508, 35.2492,10)

Map.add_basemap('HYBRID')
Map.addLayer(Classified_imgs[3] ,{'min':0,'max':1},'origin')
Map.addLayer(Iter_temporal_check_instaces[1][3] ,{'min':0,'max':1},'Iter_1')
Map.addLayer(Iter_temporal_check_instaces[2][3] ,{'min':0,'max':1},'Iter_2')

Map

Map(center=[35.2492, 115.4508], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButt…